# Multivariate linear regression
using python package statsmodels

In [2]:
import statsmodels.api as sm
import pandas as pd

Preparing Your Data: Make sure your data is in a suitable format. Typically, you'll have a pandas DataFrame with one column as the dependent variable (the outcome you're interested in predicting) and the other columns as independent variables (the predictors).

In [7]:
DATA_FOLDER = "../data/final/"
ihd_df = pd.read_csv(DATA_FOLDER + "gbd_IschemicHeartDisease_DeathsIncidence.csv")
ihd_df = ihd_df[ihd_df['measure_name'] == 'Incidence']
ihd_df

,measure_name,location_name,cause_name,year,Value,upper,lower
6180,Incidence,Samoa,Ischemic heart disease,1990,105.415246,120.256198,91.170105
6181,Incidence,Samoa,Ischemic heart disease,1991,106.893237,122.099620,92.535224
6182,Incidence,Samoa,Ischemic heart disease,1992,108.300825,124.031474,93.739971
6183,Incidence,Samoa,Ischemic heart disease,1993,109.719828,125.526585,95.309143
6184,Incidence,Samoa,Ischemic heart disease,1994,111.149369,126.996050,96.416925
...,...,...,...,...,...,...,...
12355,Incidence,Monaco,Ischemic heart disease,2015,414.483066,480.150553,353.668307
12356,Incidence,Monaco,Ischemic heart disease,2016,415.230774,481.654755,356.698254
12357,Incidence,Monaco,Ischemic heart disease,2017,417.529883,483.928186,358.608463
12358,Incidence,Monaco,Ischemic heart disease,2018,421.057297,487.301826,362.010349


In [8]:
fat_df = pd.read_csv(DATA_FOLDER + "daily_per_capita_fat_supply_final.csv")
fat_df

,Country Name,Country Code,Series Name,Year,Value
0,Afghanistan,AFG,Fat consumption per day per capita (grams),1961,38.209236
1,Afghanistan,AFG,Fat consumption per day per capita (grams),1962,38.335213
2,Afghanistan,AFG,Fat consumption per day per capita (grams),1963,39.338820
3,Afghanistan,AFG,Fat consumption per day per capita (grams),1964,39.747234
4,Afghanistan,AFG,Fat consumption per day per capita (grams),1965,40.560085
...,...,...,...,...,...
12355,Zimbabwe,ZWE,Fat consumption per day per capita (grams),2016,66.536940
12356,Zimbabwe,ZWE,Fat consumption per day per capita (grams),2017,62.848648
12357,Zimbabwe,ZWE,Fat consumption per day per capita (grams),2018,64.250500
12358,Zimbabwe,ZWE,Fat consumption per day per capita (grams),2019,56.008080


In [10]:
alcohol_df = pd.read_csv(DATA_FOLDER + "wdi_AlcoholConsumption.csv")
alcohol_df

,Country Name,Country Code,Series Name,Year,Value
0,Afghanistan,AFG,Total alcohol consumption per capita (liters o...,2000,0.00277
1,Albania,ALB,Total alcohol consumption per capita (liters o...,2000,4.43100
2,Algeria,DZA,Total alcohol consumption per capita (liters o...,2000,0.30323
3,Andorra,AND,Total alcohol consumption per capita (liters o...,2000,12.54881
4,Angola,AGO,Total alcohol consumption per capita (liters o...,2000,5.18632
...,...,...,...,...,...
926,"Venezuela, RB",VEN,Total alcohol consumption per capita (liters o...,2019,3.01625
927,Viet Nam,VNM,Total alcohol consumption per capita (liters o...,2019,9.33984
928,"Yemen, Rep.",YEM,Total alcohol consumption per capita (liters o...,2019,0.02778
929,Zambia,ZMB,Total alcohol consumption per capita (liters o...,2019,3.85176


Fitting the Model:

In [ ]:
# Assuming 'df' is your DataFrame and 'y' is the dependent variable
X = df.drop('y', axis=1)  # Independent variables
y = df['y']  # Dependent variable

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

In [ ]:
print(model.summary())